## JaneStreet Market Prediction

## Idea:Encoder + MLP
The idea of using an encoder is the denoise the data. After many attempts at using a unsupervised autoencoder, the choice landed on a bottleneck encoder as this will preserve the intra-feature relations. 

In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import numpy as np
import pandas as pd
import datatable as dt
from sklearn.model_selection import GroupKFold

from tqdm import tqdm
from random import choices

## Data Loading

In [ ]:
#  Init Random seed
SEED = 1111
np.random.seed(SEED)
tf.random.set_seed(SEED)

train_datatable = dt.fread('/kaggle/input/jane-street-market-prediction/train.csv')  # datatable perform better
train = train_datatable.to_pandas()
del train_datatable
print('Data Loaded!')
#display(train)

## Preprocess Data

In [ ]:
print('Data preprocessing...')

#limit memory use
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns})

# Remove the first 85th date due to the abnormal behavior from the laters
train = train.query('date > 85').reset_index(drop = True)   
train = train[train['weight'] != 0]

train.fillna(train.mean(),inplace=True)  # try expanding mean later

print(f"Train Share{train.shape}")
print('Done!')

## Normalization

In [ ]:
# z-score
# Pre-calc the features' mean and std for using in prediction
print('Normalization...')

features_mean = train.loc[:, train.columns.str.contains('feature')].mean()
features_std  = train.loc[:, train.columns.str.contains('feature')].std()
train_features_normorlization = (train.loc[:, train.columns.str.contains('feature')] - features_mean) / features_std

print('【train_features_normorlization】【features_mean】【features_std】Done!')

## One Hot

In [ ]:
from sklearn.preprocessing import OneHotEncoder
print('one-hot...')

# feature0: Action Assumed
f0_onehot_encoder = OneHotEncoder()
print('fitting...', end='')
f0_onehot_encoder.fit(train[['feature_0']])
print('Done!\ntransforming...', end='')
f0_onehot = f0_onehot_encoder.transform(train[['feature_0']]).toarray()
print('Feature0(Action) one-hot Done!')

## Featuring....
1. Data features original
2. f0：action onehot
3. explore the feature and tag relationship

In [ ]:
print('Merge and drop features...')

train_nparray = np.delete(train_features_normorlization.values, [0,], axis=1)

print('【train_nparray】Done!')

## Y, X Defination

In [ ]:
print('Define X and y...')

X = np.concatenate([train_nparray, f0_onehot],axis=1)

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']
y = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T

del train_nparray, f0_onehot, train_features_normorlization

print('【X_train】【y_train】Done!')

## PurgedGroupTimeSeriesSplit
Click the code button to see. 

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

### Creating the autoencoder. 
The autoencoder should aid in denoising the data. Based on [this](https://www.semanticscholar.org/paper/Deep-Bottleneck-Classifiers-in-Supervised-Dimension-Parviainen/fb86483f7573f6430fe4597432b0cd3e34b16e43) paper. 

In [ ]:
def create_autoencoder(input_dim,output_dim,noise=0.05):
    i = Input(input_dim)
    encoded = BatchNormalization()(i)
    encoded = GaussianNoise(noise)(encoded)
    encoded = Dense(64,activation='relu')(encoded)
    decoded = Dropout(0.2)(encoded)
    decoded = Dense(input_dim,name='decoded')(decoded)
    x = Dense(32,activation='relu')(decoded)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(32,activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)    
    x = Dense(output_dim,activation='sigmoid',name='label_output')(x)
    
    encoder = Model(inputs=i,outputs=encoded)
    autoencoder = Model(inputs=i,outputs=[decoded,x])
    
    autoencoder.compile(optimizer=Adam(0.005),loss={'decoded':'mse','label_output':'binary_crossentropy'})
    return autoencoder, encoder

### Creating the MLP. 

In [ ]:
import tensorflow_addons as tfa

def create_model(input_dim,output_dim,encoder):
    inputs = Input(input_dim)
    
    x = encoder(inputs)
    x = Concatenate()([x,inputs]) #use both raw and encoded features
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    hidden_units = [384, 896, 896, 394]
    for idx, hidden_unit in enumerate(hidden_units):
        x = Dense(hidden_unit)(x)
        x = BatchNormalization()(x)
        #x = Lambda(tf.keras.activations.relu)(x)
        x = Lambda(tf.keras.activations.swish)(x)
        x = Dropout(0.2)(x)
    x = Dense(output_dim,activation='sigmoid')(x)
    model = Model(inputs=inputs,outputs=x)
    model.compile(
        optimizer = tfa.optimizers.RectifiedAdam(learning_rate=1e-3),
        loss = BinaryCrossentropy(label_smoothing=1e-2),
        metrics   = tf.keras.metrics.AUC(name="auc"),
    )
    return model

### Defining and training the autoencoder. 
We add gaussian noise with mean and std from training data. After training we lock the layers in the encoder from further training. 

In [ ]:
# set TRAINING true when training
TRAINING = True

# set USE_FINETURE to turen on the fine-tune
USE_FINETUNE = True

In [ ]:
autoencoder, encoder = create_autoencoder(X.shape[-1],y.shape[-1],noise=0.1)
if TRAINING:
    autoencoder.fit(X,(X,y),
                    epochs=1000,
                    batch_size=4096, 
                    validation_split=0.1,
                    callbacks=[EarlyStopping('val_loss',patience=10,restore_best_weights=True)])
    encoder.save_weights('./encoder.hdf5')
else:
    encoder.load_weights('./encoder.hdf5')
encoder.trainable = False

### Running CV
Following [this notebook](https://www.kaggle.com/gogo827jz/jane-street-ffill-xgboost-purgedtimeseriescv) which use 5 PurgedGroupTimeSeriesSplit split on the dates in the training data. 

We add the locked encoder as the first layer of the MLP. This seems to help in speeding up the submission rather than first predicting using the encoder then using the MLP. 

We use a Baysian Optimizer to find the optimal HPs for out model. 20 trials take about 2 hours on GPU. 

In [ ]:
import matplotlib.pyplot as plt
def plot_learning(history):
    # Accuracy plot
    plt.plot(history.history['auc'])
    plt.plot(history.history['val_auc'])
    plt.title('Model accuracy')
    plt.ylabel('AUC')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Loss plot
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

In [ ]:
FOLDS = 4
INPUT = X.shape[1]
TARGET_NUM = 5   # 5 resp output
models = []

if TRAINING:
    gkf = PurgedGroupTimeSeriesSplit(n_splits = FOLDS, group_gap=20)
    splits = list(gkf.split(y, groups=train['date'].values))

    for fold, (train_indices, test_indices) in enumerate(splits):
        print(f"Fold#:{fold}")
        model = create_model(INPUT, TARGET_NUM, encoder)
        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]
        history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=4096,callbacks=[EarlyStopping('val_auc',mode='max',patience=10,restore_best_weights=True)])
        model.save_weights(f'./model_{SEED}_{fold}.hdf5')
        # Fine Tune
        if USE_FINETUNE:
            model.compile(Adam(0.00001),loss='binary_crossentropy')
            model.fit(X_test,y_test,epochs=3,batch_size=4096)
            model.save_weights(f'./model_{SEED}_{fold}_finetune.hdf5')
        models.append(model)
        plot_learning(history)

else:
    for f in range(FOLDS):
        model = create_model(INPUT, TARGET_NUM, encoder)
        if USE_FINETUNE:
            model.load_weights(f'./model_{SEED}_{f}_finetune.hdf5')
        else:
            model.load_weights(f'./model_{SEED}_{f}.hdf5')
        models.append(model)
    

## Submission

In [ ]:
models = models[-2:]
import janestreet
env = janestreet.make_env()
THRESHOLD = 0.502
f = np.median

for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        
        test_features_nparray = test_df.loc[:, test_df.columns.str.contains('feature')].values  
        
        # replace NaN with train set mean
        if np.isnan(test_features_nparray.sum()):         
            test_features_nparray = np.nan_to_num(test_features_nparray) + np.isnan(test_features_nparray) * features_mean.values
        
        # Normalization
        test_features_nparray = (test_features_nparray[0] - features_mean.values) / features_std.values
        
        # one-hot：action
        test_f0_onehot_nparray = f0_onehot_encoder.transform(test_df[['feature_0']]).toarray()
        
        
        # drop and merge
        test_features_nparray = np.delete([test_features_nparray], [0,], axis=1)
        test_features_nparray = np.concatenate([test_features_nparray, test_f0_onehot_nparray],axis=1)
        
        pred = model(test_features_nparray, training = False).numpy()
        pred = np.mean([model(test_features_nparray, training = False).numpy() for model in models],axis=0)
       
       
        # Prediction
        pred = f(pred)
        pred_df.action = np.where(pred >= THRESHOLD, 1, 0).astype(int)
        
    else:
        pred_df.action = 0
    env.predict(pred_df)